In [3]:
import tensorflow as tf


In [4]:
from object_detection.utils import label_map_util
from object_detection.utils import dataset_util
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
import keras.optimizers
from keras.regularizers import l2

# Load the tfrecord file
raw_dataset = tf.data.TFRecordDataset('enemyHead_train.tfrecord')
# Load the validation tfrecord file
valid_raw_dataset = tf.data.TFRecordDataset('enemyHead_valid.tfrecord')

# Load the label map
label_map = label_map_util.load_labelmap('enemyHead_label_map.pbtxt')
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=2, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Parse the tfrecord file
image_feature_description = {
    'image/encoded': tf.io.FixedLenFeature([], tf.string),
    'image/object/class/text': tf.io.VarLenFeature(tf.string),
    'image/object/class/label': tf.io.VarLenFeature(tf.int64),
}


def _parse_image_function(example_proto):
    # Parse the input tf.Example proto using the dictionary above.
    parsed_features = tf.io.parse_single_example(example_proto, image_feature_description)
    image = tf.image.decode_jpeg(parsed_features['image/encoded'], channels=3)
    image = tf.image.resize(image, [416, 416])
    label = tf.sparse.to_dense(parsed_features['image/object/class/label'])
    return image, label

parsed_image_dataset = raw_dataset.map(_parse_image_function).batch(32)
parsed_valid_image_dataset = valid_raw_dataset.map(_parse_image_function).batch(32)


model = Sequential()

# Convolutional layers
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(416, 416, 3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Fully connected layers
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(2, activation='softmax'))
#train this simple model(need more complex late i think)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(parsed_image_dataset, epochs=5)

train_accuracy = model.evaluate(parsed_image_dataset)
print("Training Accuracy:", train_accuracy)

valid_accuracy = model.evaluate(parsed_valid_image_dataset)
print("Validation Accuracy:", valid_accuracy)


ImportError: cannot import name 'string_int_label_map_pb2' from 'object_detection.protos' (/Users/max/anaconda3/envs/CS6476Project/lib/python3.11/site-packages/object_detection/protos/__init__.py)